# DTSA 5509 Supervised Learning Final Project
## Decision Tree Classification Tasks Applied to Food Access Research Atlas
### Project Topic
Food Deserts are areas where there is low access to supermarkets and other sources of healthy foods. They are often reported as occuring in communities that are already underserved, low income, and in Black communities. Access to food that is affordable and healthy is critical to support the health and wellbeing of a community, and areas that are struggling with poverty and inequality often also struggle with basic access to healthy food.

I took data summarizing socio-economic factors, and showed the correlation between these and low access to food through Machine Learning.  Using this socio-economic data, I explored a variety of Decision Tree classification models to attempt to classify whether a community has low access to food, using measures such as the racial composition and poverty levels as features to the models.

**For more information**

Annie E. Casey Foundation: Food Deserts in the United States: https://www.aecf.org/blog/exploring-americas-food-deserts

Wikipedia: Food Desert: https://en.wikipedia.org/wiki/Food_desert

### Imports

In [41]:
import pandas as pd
import altair as alt
import dtale
from sklearn.svm import SVC
from xgboost import XGBClassifier
from xgboost import DMatrix
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from sklearn.preprocessing import OneHotEncoder
from imblearn.under_sampling import RandomUnderSampler
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

### Data
The data used in this report is the Food Access Research Atlas, published by the Economic Research Service, Department of Agriculture, via data.gov.

This dataset has 72,531 records, one for each U.S. Census tract.  The dataset includes 147 features for each tract, describing the tract, the population of the tract, demographic breakdowns of each tract (population Black, Seniors, Children, etc), and numerous measures on the food access for each tract.

For my task, I wanted to focus on how the socio-economic factors contribututed to whether a tract was likely to be classified as low access to food.  I limited the features used to the following:

| Field              | Description                                                                                                 |
|:-------------------|:------------------------------------------------------------------------------------------------------------|
| Urban              | Flag for urban tract                                                                                        |
| OHU2010            | Occupied housing unit count from 2010 census                                                                |
| LowIncomeTracts    | Flag for low income tract                                                                                   |
| PovertyRate        | Share of the tract population living with income at or below the Federal poverty thresholds for family size |
| MedianFamilyIncome | Tract median family income                                                                                  |
| LA1and20           | Flag for low access tract at 1 mile for urban areas or 20 miles for rural areas                             |
| TractLOWI          | Total count of low-income population in tract                                                               |
| TractKids          | Total count of children age 0-17 in tract                                                                   |
| TractSeniors       | Total count of seniors age 65+ in tract                                                                     |
| TractWhite         | Total count of White population in tract                                                                    |
| TractBlack         | Total count of Black or African American population in tract                                                |
| TractAsian         | Total count of Asian population in tract                                                                    |
| TractNHOPI         | Total count of Native Hawaiian and Other Pacific Islander population in tract                               |
| TractAIAN          | Total count of American Indian and Alaska Native population in tract                                        |
| TractOMultir       | Total count of Other/Multiple race population in tract                                                      |
| TractHispanic      | Total count of Hispanic or Latino population in tract                                                       |
| TractHUNV          | Total count of housing units without a vehicle in tract                                                     |
| TractSNAP          | Total count of housing units receiving SNAP benefits in tract                                               |

Field `LA1and20` is the label I'm attempting to predict.  The remaining fields are the features.


Data: https://catalog.data.gov/dataset/food-access-research-atlas

About the Atlas: https://www.ers.usda.gov/data-products/food-access-research-atlas/about-the-atlas.aspx

In addition, I'm using the U.S. Census Divisions listed at the following URL to further supplement my data.

https://www.ncei.noaa.gov/access/monitoring/reference-maps/us-census-divisions

**Data Citations**

Economic Research Service, Department of Agriculture (2021, February 24). Data.Gov. Food Access Research Atlas. Retrieved April 23, 2023, from https://catalog.data.gov/dataset/food-access-research-atlas

In [2]:
label = 'LILATracts_1And10'

In [3]:
# Create variable table and copy to clipboard for above
used_columns = [label,'Pop2010','State','OHU2010','Urban','PovertyRate','MedianFamilyIncome','TractBlack','TractLOWI','TractKids','TractSeniors','TractWhite','TractAsian','TractNHOPI','TractAIAN','TractOMultir','TractHispanic','TractHUNV','TractSNAP']
var_df = pd.read_excel('../data/FoodAccessResearchAtlasData2019.xlsx', sheet_name='Variable Lookup')
var_df = var_df[var_df['Field'].isin(used_columns)][['Field','Description']]
#pd.io.clipboards.to_clipboard(var_df.to_markdown(index=False), excel=False)

### Data Cleaning

As outlined above, I've limited the features used to the most relevant features for the classification task I wanted to perform.  Of the 147 features, 18 are relevant for the task of classifying the low food access status based on the demographic makeup of an area.  Many of the other features in the original file describe in detail the low food access for specific populations, so have been left out since they would be auto-correlative with the low access label I'm attempting to predict.

In addition, while reviewing the number of NA values present in the data, I found that `MedianFamilyIncome` had a high count of NAs. I have dropped this feature from the dataset for this reason.

`OHU2010` has values of 0 for 106 rows. This appears to be null data, coded as 0.  As this will cause a division by 0 error when calculating percentages below, so I'm also dropping these rows.

The remaining features have a low number of NA values.  For these, I have dropped the rows (tracts) that contained NA values.

In the original dataset, the demographic data is reported as total counts of residents in that demographic. Because I want to compare tracts, and each tract may have a different total population, it makes most sense to convert these to percentages.  Below, I calculate the percentage of residents in each demographic using the demographic count and the total population count `Pop2010`.  After I've done these calculations, I drop the original count columns as they are no longer needed.

I also investigated whether the labels in the dataset were balanced.  I've found that they are not, and I rebalance these later while building the model.

#### Read Data

In [4]:
# Read Data
df = pd.read_excel('../data/FoodAccessResearchAtlasData2019.xlsx', sheet_name='Food Access Research Atlas')
df = df[used_columns]

#### Check for NAs, Vizualize, and Clean

In [5]:
# Check for NAs and vizualize
na_counts = pd.DataFrame(df.isna().sum()).reset_index()
na_counts.columns = ['Field', 'NA Count']
na_counts = na_counts.sort_values('NA Count', ascending=False)
base = alt.Chart(na_counts).encode(
    x=alt.X('Field:N', sort='-y'),
    y='NA Count',
    text = 'NA Count'
).properties(width=800)
base.mark_bar() + base.mark_text(align='center', dy=-10)

alt.LayerChart(...)

In [6]:
len_df_bef = len(df)
# Drop high NA column Median Family Income
df = df.drop('MedianFamilyIncome', axis=1)
# Drop records with NAs in values
df = df.dropna()
print(str(len_df_bef - len(df)) + ' rows dropped')
# Check for no NAs
df.isna().sum()

4 rows dropped


LILATracts_1And10    0
Pop2010              0
State                0
OHU2010              0
Urban                0
PovertyRate          0
TractBlack           0
TractLOWI            0
TractKids            0
TractSeniors         0
TractWhite           0
TractAsian           0
TractNHOPI           0
TractAIAN            0
TractOMultir         0
TractHispanic        0
TractHUNV            0
TractSNAP            0
dtype: int64

In [7]:
len_df_bef = len(df)
print(sum(df['Pop2010'] == 0))
print(sum(df['OHU2010'] == 0))
df = df[df['OHU2010'] > 0]
print(str(len_df_bef - len(df)) + ' rows dropped')
print(sum(df['Pop2010'] == 0))
print(sum(df['OHU2010'] == 0))

0
106
106 rows dropped
0
0


#### Calculate Percentage Share for Each Demographic

In [8]:
# Calculate percentage share for each demographic population
df['BlackPopShare'] = df['TractBlack'] / df['Pop2010']
df['LOWIPopShare'] = df['TractLOWI'] / df['Pop2010']
df['KidsPopShare'] = df['TractKids'] / df['Pop2010']
df['SeniorsPopShare'] = df['TractSeniors'] / df['Pop2010']
df['WhitePopShare'] = df['TractWhite'] / df['Pop2010']
df['AsianPopShare'] = df['TractAsian'] / df['Pop2010']
df['NHOPIPopShare'] = df['TractNHOPI'] / df['Pop2010']
df['AIANPopShare'] = df['TractAIAN'] / df['Pop2010']
df['OMultirPopShare'] = df['TractOMultir'] / df['Pop2010']
df['HispanicPopShare'] = df['TractHispanic'] / df['Pop2010']
df['HUNVPopShare'] = df['TractHUNV'] / df['OHU2010']
df['SNAPPopShare'] = df['TractSNAP'] / df['OHU2010']
df = df.drop(['Pop2010','OHU2010','TractBlack','TractLOWI','TractKids','TractSeniors','TractWhite','TractAsian','TractNHOPI','TractAIAN','TractOMultir','TractHispanic','TractHUNV','TractSNAP'], axis=1)

#### Add U.S Census Divisions to Data

In [9]:
div_map = {
    'Illinois':'East North Central',
    'Indiana':'East North Central',
    'Michigan':'East North Central',
    'Ohio':'East North Central',
    'Wisconsin':'East North Central',
    'Alabama':'East South Central',
    'Kentucky':'East South Central',
    'Mississippi':'East South Central',
    'Tennessee':'East South Central',
    'New Jersey':'Middle Atlantic',
    'New York':'Middle Atlantic',
    'Pennsylvania':'Middle Atlantic',
    'Arizona':'Mountain',
    'Colorado':'Mountain',
    'Idaho':'Mountain',
    'Montana':'Mountain',
    'New Mexico':'Mountain',
    'Nevada':'Mountain',
    'Utah':'Mountain',
    'Wyoming':'Mountain',
    'Connecticut':'New England',
    'Maine':'New England',
    'Massachusetts':'New England',
    'New Hampshire':'New England',
    'Rhode Island':'New England',
    'Vermont':'New England',
    'California':'Pacific',
    'Oregon':'Pacific',
    'Washington':'Pacific',
    'Delaware':'South Atlantic',
    'Florida':'South Atlantic',
    'Georgia':'South Atlantic',
    'Maryland':'South Atlantic',
    'North Carolina':'South Atlantic',
    'South Carolina':'South Atlantic',
    'Virginia':'South Atlantic',
    'West Virginia':'South Atlantic',
    'Iowa':'West North Central',
    'Kansas':'West North Central',
    'Minnesota':'West North Central',
    'Missouri':'West North Central',
    'Nebraska':'West North Central',
    'North Dakota':'West North Central',
    'South Dakota':'West North Central',
    'Arkansas':'West South Central',
    'Louisiana':'West South Central',
    'Oklahoma':'West South Central',
    'Texas':'West South Central'
}
df['Census Division'] = df['State'].map(div_map)

#### Check for Label Imbalance

In [10]:
# Check for imbalaced labels
val_counts = pd.DataFrame(df[label].value_counts()).reset_index()
base = alt.Chart(val_counts).encode(
    x=alt.X(label + ':N', sort='-y'),
    y='count',
    text = 'count'
).properties(width=100)
base.mark_bar(size=30) + base.mark_text(align='center', dy=-10)

alt.LayerChart(...)

### Exploratory Data Analysis

In [11]:
eda_df = df.copy()
eda_df = eda_df.drop(['State','Census Division'], axis=1)
cor_data = (eda_df.corr().stack().reset_index() 
              .rename(columns={0: 'correlation', 'level_0': 'variable', 'level_1': 'variable2'}))
cor_data['correlation_label'] = cor_data['correlation'].map('{:.2f}'.format)

In [12]:
base = alt.Chart(cor_data).encode(
    x='variable2:O',
    y='variable:O'    
)

text = base.mark_text().encode(
    text='correlation_label',
    color=alt.condition(
        alt.datum.correlation > 0.5, 
        alt.value('white'),
        alt.value('black')
    )
)

cor_plot = base.mark_rect().encode(
    color=alt.Color('correlation:Q', scale=alt.Scale(scheme='redblue'))
).properties(width=600, height=600)

cor_plot + text

alt.LayerChart(...)

In [13]:
bins = [0, .1, .2, .3, .4, .5, .6, .7, .8, .9, 1.0]
names = ['0%-10%', '10%-20%', '20%-30%', '30%-40%', '40%-50%', '50%-60%', '60%-70%', '70%-80%', '80%-90%', '90%-100%']

In [14]:
eda_df = df.copy()
eda_df['PopShareRange'] = pd.cut(eda_df['BlackPopShare'], bins, labels=names)
b = eda_df[['PopShareRange',label]].groupby(['PopShareRange'], as_index=False).aggregate({label:['sum','count', 'mean']})
b.columns = [f"{x}_{y}" for x, y in b.columns.to_flat_index()]
b['Demographic'] = 'Black'

eda_df['PopShareRange'] = pd.cut(eda_df['WhitePopShare'], bins, labels=names)
w = eda_df[['PopShareRange',label]].groupby(['PopShareRange'], as_index=False).aggregate({label:['sum','count', 'mean']})
w.columns = [f"{x}_{y}" for x, y in w.columns.to_flat_index()]
w['Demographic'] = 'White'

bw = pd.concat([b,w])

In [15]:
alt.Chart(bw).mark_bar(size=25).encode(
    x=alt.X('Demographic:N', title='', axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('LILATracts_1And10_mean:Q', axis=alt.Axis(format='%'), title='Percent Low Income, Low Access Tracts'),
    color='Demographic:N',
    column=alt.Column('PopShareRange_:O', title='Population Percentage Range')
).properties(
    height=300,
    width=60
).configure_axis(
    labelFontSize=13,
    titleFontSize=15
).configure_header(
    labelFontSize=13,
    titleFontSize=15,
    labelOrient='bottom',
    titleOrient='bottom'
)

alt.Chart(...)

In [16]:
eda_df = df.copy()
eda_df['PopShareRange'] = pd.cut(eda_df['KidsPopShare'], bins, labels=names)
k = eda_df[['PopShareRange',label]].groupby(['PopShareRange'], as_index=False).aggregate({label:['sum','count', 'mean']})
k.columns = [f"{x}_{y}" for x, y in k.columns.to_flat_index()]

In [17]:
alt.Chart(k).mark_bar(size=25).encode(
    x=alt.X('PopShareRange_:O', title='Population Percentage Range', axis=alt.Axis(labelAngle=-45)),
    y=alt.Y('LILATracts_1And10_mean:Q', axis=alt.Axis(format='%'), title='Percent Low Income, Low Access Tracts'),
).properties(
    height=300,
    width=600
).configure_axis(
    labelFontSize=13,
    titleFontSize=15
)

alt.Chart(...)

In [18]:
eda_df = df.copy()
k = eda_df[['State',label]].groupby(['State'], as_index=False).aggregate({label:['sum','count', 'mean']})
k.columns = [f"{x}_{y}" for x, y in k.columns.to_flat_index()]

In [19]:
alt.Chart(k).mark_bar(size=10).encode(
    y=alt.Y('State_:N', title='State', sort='-x'),
    x=alt.X('LILATracts_1And10_mean:Q', axis=alt.Axis(format='%'), title='Percent Low Income, Low Access Tracts'),
).properties(
    height=800,
    width=600
).configure_axis(
    labelFontSize=13,
    titleFontSize=15
)

alt.Chart(...)

In [20]:
eda_df = df.copy()
d = eda_df[['Census Division',label]].groupby(['Census Division'], as_index=False).aggregate({label:['sum','count', 'mean']})
d.columns = [f"{x}_{y}" for x, y in d.columns.to_flat_index()]

In [21]:
alt.Chart(d).mark_bar(size=20).encode(
    y=alt.Y('Census Division_:N', title='Census Division', sort='-x'),
    x=alt.X('LILATracts_1And10_mean:Q', axis=alt.Axis(format='%'), title='Percent Low Income, Low Access Tracts'),
).properties(
    height=300,
    width=600
).configure_axis(
    labelFontSize=13,
    titleFontSize=15
)

alt.Chart(...)

### Models

#### Test/Train Preparation

In [22]:
data = df.loc[:, df.columns.isin(['PovertyRate','MedianFamilyIncome','LowIncomeTracts','BlackPopShare','LOWIPopShare','KidsPopShare','SeniorsPopShare','WhitePopShare','AsianPopShare','NHOPIPopShare','AIANPopShare','OMultirPopShare','HispanicPopShare','HUNVPopShare','SNAPPopShare'])]
data = pd.concat([data, pd.get_dummies(df['Urban'])], axis=1)
data = data.rename({0:'Rural', 1:'Urban'}, axis=1)
data = pd.concat([data, pd.get_dummies(df['Census Division'])], axis=1)
label = df[label]
X_train, X_test, y_train, y_test = train_test_split(data, label, test_size=0.2, stratify=label)
print(len(X_train), len(X_test), len(y_train), len(y_test), sum(y_train == 1), sum(y_train == 0))

57936 14485 57936 14485 7431 50505


In [23]:
under_sampler = RandomUnderSampler()

X_res, y_res = under_sampler.fit_resample(X_train, y_train)
print(len(X_res), len(X_test), len(y_res), len(y_test), sum(y_res == 1), sum(y_res == 0))

14862 14485 14862 14485 7431 7431


#### Decision Tree Classifier

In [35]:
dcs = DecisionTreeClassifier(max_depth=5, max_leaf_nodes=50)
dcs.fit(X_res, y_res)

DecisionTreeClassifier(max_depth=5, max_leaf_nodes=50)

In [36]:
y_pred = dcs.predict(X_test)

In [37]:
precision, recall, fbeta_score, support = precision_recall_fscore_support(y_test, y_pred, pos_label=1, average='binary')

In [38]:
fbeta_score

0.4180806675938804

In [39]:
accuracy_score(y_test, y_pred)

0.7111494649637556

In [40]:
roc_auc_score(y_test, y_pred)

0.752847623580376

#### AdaBoost

In [66]:
ada = AdaBoostClassifier(learning_rate=1)
ada.fit(X_res, y_res)

AdaBoostClassifier(learning_rate=1)

In [67]:
y_pred = ada.predict(X_test)

In [68]:
precision, recall, fbeta_score, support = precision_recall_fscore_support(y_test, y_pred, pos_label=1, average='binary')

In [69]:
fbeta_score

0.444639718804921

In [70]:
accuracy_score(y_test, y_pred)

0.7382119433897135

In [71]:
roc_auc_score(y_test, y_pred)

0.7718125502590131

#### XGBoost

In [ ]:
dtrain = DMatrix(X_res, label=y_res)
dtest = DMatrix(X_test, label=y_test)
evallist = [(dtrain, 'train'), (dtest, 'eval')]

In [ ]:
params = {
        'min_child_weight': [1, 5, 10],
        'gamma': [0.5, 1, 1.5, 2, 5],
        'subsample': [0.6, 0.8, 1.0],
        'colsample_bytree': [0.6, 0.8, 1.0],
        'max_depth': [5, 10, 15]
        }
xgb = XGBClassifier(learning_rate=0.02, objective='binary:logistic', nthread=1)

search = GridSearchCV(xgb, params, scoring='roc_auc', n_jobs=4, verbose=4)

search.fit(X_res, y_res)

In [ ]:
search.best_score_

In [ ]:
search.best_params_

In [ ]:
param = {'max_depth': 7, 'eta': 0.3, 'objective': 'binary:logistic', 'scale_pos_weight':sum(label) / sum(1-label)}
param['nthread'] = 4
param['eval_metric'] = 'auc'
num_round = 100
bst = xgb.train(param, dtrain, num_round, evallist, early_stopping_rounds=5)

In [ ]:
xgb.plot_importance(bst)

In [ ]:
format = 'svg' #You should try the 'svg'
image = xgb.to_graphviz(bst, num_trees=0)
image.graph_attr = {'dpi':'400'}
image.render('tree', format = format)

In [ ]:
#['PovertyRate','BlackPopShare','LOWIPopShare','KidsPopShare','SeniorsPopShare','WhitePopShare','AsianPopShare','NHOPIPopShare','AIANPopShare','OMultirPopShare','HispanicPopShare','HUNVPopShare','SNAPPopShare']
data = []
dtest = xgb.DMatrix(data)
ypred = bst.predict(dtest)